In [ ]:
import numpy as np
import numba
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.integrate import solve_ivp
from plotting import plot
from typing import Callable


In [ ]:
STANDARD_GRAV = 9.80665
Cd = 0.8
S_ref = 10
thrust = 2100000.0
g0 = STANDARD_GRAV
Isp = 350
#a0 = -0.001
#b0 = 3.0
config_list = [
    ((0,10),None, (thrust, Isp), (0,  (np.pi/2*0.9))),
    ((None,500), None,(thrust, Isp), (1,  (None))),
]


x0 = np.array([0,0,0,0,120000])

In [ ]:

@numba.njit
def angle_steering(t: float, x: np.array , params: tuple) -> float:
    u = params
    return u
@numba.njit
def zero_alpha(t: float, x: np.array , params: tuple) -> float:
    """For zero alpha pitch = flight path angle 
    Args:
        t : time
        x : Vehicle state
        params : control law params, this law has no parameters 
    Returns:
        Desired pitch
    """
    vx, vy = x[2:4]
    return np.arctan2(vy, vx)
@numba.njit
def polynomial_steering(t: float, x: np.array , params: tuple) -> float:
    a,b,c = params
    return a*t**2 + b*t + c
@numba.njit
def lts_control(t: float, x: np.array , params: tuple) -> float:
    a,b = params
    return np.arctan(a*t+ b)


In [ ]:
#@numba.njit
def control(t: float, x: np.array, params:tuple) -> float:
    """Functions selects the parameterized control scheme and returns the desired pitch angle

    Args:
        t : time
        x : Vehicle state
        params : Tuple of parameters containing the control type and control law parameters

    Returns:
        Desired pitch
    """
    ctrl_type, ctrl_param= params
    if ctrl_type == 0:
        return angle_steering(t, x, ctrl_param)
    elif ctrl_type == 1:
        return zero_alpha(t, x, ctrl_param)
    elif ctrl_type == 2:
        return polynomial_steering(t, x, ctrl_param)
    elif ctrl_type == 3:
        return lts_control(t, x, ctrl_param)
    else:
        print("Control law not define")

In [ ]:
#@numba.njit
def dynamics(t, x, params):
    Cd = 0.8
    S_ref = 10
    thrust, Isp = params[0]
    u = control(t, x, params[1])
    cos_theta  = np.cos(u)
    sin_theta  = np.sin(u)
    dx = np.zeros_like(x)
    dx[0] = x[2]
    dx[1] = x[3]
    dx[2] =  (thrust * cos_theta ) / x[4] 
    dx[3] =  (thrust * sin_theta )  / x[4] - g0
    dx[4] =  -thrust / STANDARD_GRAV / Isp
    return dx

In [ ]:
def mass_limit(t, x, params):
    return x[4] - 1000

mass_limit.terminal = True
mass_limit.direction = -1  # Terminate when value crosses zero

$$ q={\frac {1}{2}}\rho \,u^{2} $$


In [ ]:
@lru_cache(maxsize=128, typed=True)
def problem_rollout(x0: np.array, config_list: list):
    arr = tuple(x0)
    t_init= 0.0
    for configs in config_list:
        print(configs)
        t_span, event, params = configs
        sol = solve_ivp(
            dynamics, 
            t_span=[t_init, t_span[1]], 
            y0=x0,    
            args=(params,), 
            events=mass_limit)
        x0 = sol.y[-1]
        t_init = sol.t[-1]
        print(t_init)
    return sol  

In [93]:
# ((t_start,tend), (vch properties, (phase_id, (phase_params)))
config_list = [
    ((0,10),None, ((thrust, Isp), (0,  (np.pi/2*0.9)))),
    ((None,500), None,((thrust, Isp), (1,  (None)))),
    ((None,1000), None,((thrust, Isp), (1,  (None)))),

]

In [94]:
sol = problem_rollout(x0, config_list)

TypeError: unhashable type: 'numpy.ndarray'

In [ ]:
plot(
    sol.t,[sol.y[0], sol.y[1]], y2 = [sol.y[2], sol.y[3]],
    title="Time vs States", 
    xlabel="Time", 
    ylabel=("Pos", "Vel"),
    trace_names=("pos_x", "pos_y", "vel_x", "vel_y")
    )

In [ ]:
plot(
    sol.t,[sol.y[4]],
    title="Time vs Mass", 
    xlabel="Time", 
    ylabel=("Mass"),
    )

In [ ]:
mdot = thrust / STANDARD_GRAV / Isp

t_max = 50000 / mdot

In [ ]:
plot(
    sol.t,[sol.y[0], sol.y[1]], y2 = [sol.y[2], sol.y[3]],
    title="Time vs States", 
    xlabel="Time", 
    ylabel=("Pos", "Vel"),
    trace_names=("pos_x", "pos_y", "vel_x", "vel_y")
    )

In [ ]:
plot( sol.t, [sol.y[4]], title="time vs Mass", xlabel="Time", ylabel="Mass")

In [95]:
import numpy as np
from functools import lru_cache

@lru_cache(maxsize=None)
def my_func(arr):
    arr = tuple(arr)  # Convert array to tuple
    

result = my_func(np.array([1, 2, 3])) 

TypeError: unhashable type: 'numpy.ndarray'

In [ ]:
multiply_by_two.cache_info()

In [ ]:
expensive.count